In [1]:
import sys
sys.path.append('../')
import jax.numpy as jnp

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
import src.models.rnn as rnn
import jax

In [3]:
trf=rnn.rnn_transform(rnn.MultiplicativeRNN,10,4,32)

In [4]:
key=jax.random.PRNGKey(0)
sample_o=jax.random.normal(key,[20,10])
sample_a=jax.random.normal(key,[20,4])
h_t=rnn.MultiplicativeRNN.initial_state(10,4,32,4)
rnn_params=trf.init(key,(sample_o[0],sample_a[0]),h_t)

In [5]:
for i in range(12):
    out,h_t=(trf.apply)(rnn_params,(sample_o[i],sample_a[i]),h_t)

In [10]:
def hvp(rnn_params,inputs,h_t,weights):
    def jvp(rnn_params,inputs,h_t,weights):
        jacobians=jax.jacrev(trf.apply)(rnn_params,(sample_o[0],sample_a[0]),h_t)[0]
        jvp=jax.tree_multimap(lambda x, y: jnp.tensordot(x,y,axes=y.ndim), jacobians, weights)
        jvp=jnp.stack(jax.tree_util.tree_flatten(jvp)[0],axis=0).sum(axis=0)
        return jvp
    hvp_fn=jax.jacrev(jvp)
    return hvp_fn(rnn_params,inputs,h_t,weights)

In [11]:
hvp=jax.jit(hvp)

In [12]:
import time

In [18]:
start=time.time()
hvp(rnn_params,(sample_o[0],sample_a[0]),h_t,rnn_params)
time.time()-start

0.07254195213317871

In [7]:
sample_o[0],sample_a[0]

(DeviceArray([-2.5344267 ,  0.986253  , -0.28956777,  0.58931786,
              -0.31085405,  1.695505  ,  0.7945188 ,  1.7130398 ,
              -1.2360951 , -0.43915448], dtype=float32),
 DeviceArray([-0.5157282,  0.6021314, -1.5149792,  0.723516 ], dtype=float32))

In [8]:
a=jax.jacrev(trf.apply)(rnn_params,(sample_o[0],sample_a[9]),h_t)

In [9]:
a

(FlatMap({
   'multiplicative_rnn': FlatMap({
                           'b': DeviceArray([[[ 1.8185683e-01, -1.0788087e-01, -4.9009693e-01,
                                               -1.7953689e-01],
                                              [ 1.8624751e-03, -1.2218066e-03,  4.8456606e-04,
                                               -2.5808464e-03],
                                              [-4.6850857e-03,  7.7714591e-04, -5.9781559e-03,
                                                8.8101542e-03],
                                              ...,
                                              [ 3.8536231e-03, -3.8559752e-04,  5.3545404e-03,
                                               -7.4894130e-03],
                                              [-6.6740415e-03,  2.2962196e-03, -6.1488096e-03,
                                                1.1364515e-02],
                                              [ 6.4982129e-03, -1.8503007e-03,  6.9491826e-03,
             